# LightMem Example with longmemeval data

Tutorial author: xubuqiang

## 0. Prepare the runtime environment

In [ ]:
# Set your LightMemory project path
LIGHTMEM_PROJECT_PATH = '/disk/disk_20T/xubuqiang/lightmem'

# Install in editable mode
%cd {LIGHTMEM_PROJECT_PATH}
%env ALL_PROXY=
!pip install -e .
%cd tutorial-notebooks

/disk/disk_20T/xubuqiang/lightmem
env: ALL_PROXY=
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///disk/disk_20T/xubuqiang/lightmem
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/5a/63/4fdc45a0304536e75a5e1b1bbfb1b56dd0e2743c48ee83ca729f7ce44162/torch-2.8.0-cp311-cp311-manylinux_2_28_x86_64.whl (888.1 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/e5/2b/4d2708ac1ff5cd708b6548f4c5812d0ae40d1c28591c4c1c762b6dbdef2d/transformers-4.57.0-py3-none-any.whl (12.0 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/48/21/4670d03ab8587b0ab6f7d5fa02a95c3dd6b1f39d0e40e508870201f3d76c/sentence_transformers-5.1.1-py3-none-any.whl (486 kB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/5f/a0/d9ef19f780f319c21ee90ecfef443

## 1. Import Dependencies

In [1]:
import os
import json
import datetime
from lightmem.memory.lightmem import LightMemory
from typing import List, Dict, Any
import pandas as pd
from tqdm import tqdm

In [ ]:
# logging setup
LOGS_ROOT = "./logs"
RUN_TIMESTAMP = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_LOG_DIR = os.path.join(LOGS_ROOT, RUN_TIMESTAMP)
os.makedirs(RUN_LOG_DIR, exist_ok=True)

# API
API_KEY = ''
API_BASE_URL = ''
LLM_MODEL = 'gpt-4o-mini'

LLMLINGUA_MODEL_PATH = '/disk/disk_20T/fangjizhan/models/llmlingua-2-bert-base-multilingual-cased-meetingbank'
EMBEDDING_MODEL_PATH = '/disk/disk_20T/fangjizhan/models/all-MiniLM-L6-v2'
DATA_FILE_PATH = '/longmemeval_single.json'

print(f"RUN_LOG_DIR: {RUN_LOG_DIR}")
print(f"DATA_FILE_PATH: {DATA_FILE_PATH}")

RUN_LOG_DIR: ./logs/20251206_202021
DATA_FILE_PATH: /disk/disk_20T/xubuqiang/lightmem/dataset/longmemeval/longmemeval_single.json


## 2. LightMemory Initial config

In [4]:
config_dict = {
    "pre_compress": True,
    "pre_compressor": {
        "model_name": "llmlingua-2",
        "configs": {
            "llmlingua_config": {
                "model_name": LLMLINGUA_MODEL_PATH,
                "device_map": "cuda",
                "use_llmlingua2": True,
            },
        }
    },
    "topic_segment": True,
    "precomp_topic_shared": True,
    "topic_segmenter": {
        "model_name": "llmlingua-2",
    },
    "messages_use": "user_only",
    "metadata_generate": True,
    "text_summary": True,
    "memory_manager": {
        "model_name": 'openai',
        "configs": {
            "model": LLM_MODEL,
            "api_key": API_KEY,
            "max_tokens": 16000,
            "openai_base_url": API_BASE_URL
        }
    },
    "extract_threshold": 0.1,
    "index_strategy": "embedding",
    "text_embedder": {
        "model_name": "huggingface",
        "configs": {
            "model": EMBEDDING_MODEL_PATH,
            "embedding_dims": 384,
            "model_kwargs": {"device": "cuda"},
        },
    },
    "retrieve_strategy": "embedding",
    "embedding_retriever": {
        "model_name": "qdrant",
        "configs": {
            "collection_name": "longmemeval_demo",
            "embedding_model_dims": 384,
            "path": "./longmemeval_demo_db", 
        }
    },
    "update": "offline",
    "logging": {
        "level": "DEBUG",
        "file_enabled": True,
        "log_dir": RUN_LOG_DIR,
    }
}

print("Initial LightMem...")
lightmem = LightMemory.from_config(config_dict)
print("LightMem initialized!")

Initial LightMem...
pre_compressor:llmlingua-2
pre_compressor:llmlingua_config={'model_name': '/disk/disk_20T/fangjizhan/models/llmlingua-2-bert-base-multilingual-cased-meetingbank', 'device_map': 'cuda', 'use_llmlingua2': True} llmlingua2_config={'max_batch_size': 50, 'max_force_token': 100} compress_config={'instruction': '', 'rate': 0.8, 'target_token': -1}


2025-12-06 20:20:25 - LightMemory - INFO - Initializing LightMemory with provided configuration
2025-12-06 20:20:25 - LightMemory - INFO - Token statistics tracking initialized
2025-12-06 20:20:25 - LightMemory - INFO - Initializing pre-compressor
`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-06 20:21:15 - LightMemory - INFO - Initializing topic segmenter
2025-12-06 20:21:15 - LightMemory - INFO - Initializing memory manager


DEBUG: resolved to encoding o200k_base


2025-12-06 20:21:16 - LightMemory - INFO - Initializing text embedder
2025-12-06 20:21:16 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: /disk/disk_20T/fangjizhan/models/all-MiniLM-L6-v2


ShortMemBufferManager initialized with max_tokens=512


2025-12-06 20:21:16 - LightMemory - INFO - Initializing embedding retriever
2025-12-06 20:21:16 - LightMemory - INFO - LightMemory initialization completed successfully


LightMem initialized!


## 3. Load dataset


In [5]:
with open(DATA_FILE_PATH, 'r', encoding='utf-8') as f:
    data = json.load(f)

if isinstance(data, list):
    data_item = data[0]
else:
    data_item = data

question_ids = [item.get('question_id', '') for item in data]
question_types = [item.get('question_type', '') for item in data]
questions = [item.get('question', '') for item in data]
question_dates = [item.get('question_date', '') for item in data]
answers = [item.get('answer', '') for item in data]
answer_session_ids = [item.get('answer_session_ids', []) for item in data]

# Collect all unique haystack sessions (avoid duplicates)
haystack_sessions_dict = {}
haystack_dates_dict = {}

for item in data:
    sessions = item.get('haystack_sessions', [])
    dates = item.get('haystack_dates', [])
    session_ids = item.get('haystack_session_ids', [])
    
    for session, date, sid in zip(sessions, dates, session_ids):
        if sid not in haystack_sessions_dict:
            haystack_sessions_dict[sid] = session
            haystack_dates_dict[sid] = date

haystack_session_ids = list(haystack_sessions_dict.keys())
haystack_sessions = list(haystack_sessions_dict.values())
haystack_dates = list(haystack_dates_dict.values())
print(f"\nDataset Statistics:")
print(f"- Number of questions: {len(questions)}")
print(f"- Number of unique haystack sessions: {len(haystack_sessions)}")
print(f"- Haystack session IDs: {haystack_session_ids[:5]}..." if len(haystack_session_ids) > 5 else f"- Haystack session IDs: {haystack_session_ids}")

print(f"\nQuestion Preview:")
for i, (qid, q) in enumerate(zip(question_ids[:3], questions[:3])):
    preview = q[:100] + '...' if len(q) > 100 else q
    print(f"  [{qid}] {preview}")

print(f"\nQuestion Types Distribution:")
from collections import Counter
type_counts = Counter(question_types)
for qtype, count in type_counts.items():
    print(f"  {qtype}: {count}")


Dataset Statistics:
- Number of questions: 1
- Number of unique haystack sessions: 53
- Haystack session IDs: ['sharegpt_yywfIrx_0', '85a1be56_1', 'sharegpt_Jcy1CVN_0', 'sharegpt_Cr2tc1f_0', 'sharegpt_DGTCD7D_0']...

Question Preview:
  [e47becba] What degree did I graduate with?

Question Types Distribution:
  single-session-user: 1


## 4. ADD memory into LightMem


In [ ]:
METADATA_GENERATE_PROMPT = """
You are a Personal Information Extractor. 
Your task is to extract **all possible facts or information** about the user from a conversation, 
where the dialogue is organized into topic segments separated by markers like:

Input format:
--- Topic X ---
[timestamp, weekday] source_id.SpeakerName: message
...

Important Instructions:
0. You MUST process messages **strictly in ascending sequence_number order** (lowest → highest). For each message, stop and **carefully** evaluate its content before moving to the next. Do NOT reorder, batch-skip, or skip ahead — treat messages one-by-one.
1. You MUST process every user message in order, one by one. 
   For each message, decide whether it contains any factual information.
   - If yes → extract it and rephrase into a standalone sentence.
   - If no (pure greeting, filler, or irrelevant remark) → skip it.
   - Do NOT skip just because the information looks minor, trivial, or unimportant. 
     Even small details (e.g., "User drank coffee this morning") must be kept. 
     Only skip if it is *completely* meaningless (e.g., "Hi", "lol", "thanks").
2. Perform light contextual completion so that each fact is a clear standalone statement.
   Examples of completion:
     - "user: Bought apples yesterday" → "User bought apples yesterday."
     - "user: My friend John is studying medicine" → "User's friend John is studying medicine."
3. Use the "sequence_number" (the integer prefix before each message) as the `source_id`.
4. Output format:
Please return your response in JSON format.
   {
     "data": [
       {
         "source_id": "<source_id>",
         "fact": "<complete fact with ALL specific details>"
       }
     ]
   }


Examples:

--- Topic 1 ---
[2022-03-20T13:21:00.000, Sun] 0.User: My name is Alice and I work as a teacher.
[2022-03-20T13:21:00.500, Sun] 2.User: My favourite movies are Inception and Interstellar.
--- Topic 2 ---
[2022-03-20T13:21:01.000, Sun] 4.User: I visited Paris last summer.
{"data": [
  {"source_id": 0, "fact": "User's name is Alice."},
  {"source_id": 0, "fact": "User works as a teacher."},
  {"source_id": 2, "fact": "User's favourite movies are Inception and Interstellar."},
  {"source_id": 4, "fact": "User visited Paris last summer."}
]}

Reminder: Be exhaustive. Unless a message is purely meaningless, extract and output it as a fact.
"""


In [6]:
def convert_timestamp(timestamp: str) -> str:
    """
    Convert timestamp from '2025/12/02 (Tue) 17:06' to '2025-12-02 17:06:00'
    
    Args:
        timestamp: Original timestamp string
        
    Returns:
        Converted timestamp string in format '%Y-%m-%d %H:%M:%S'
    """
    from datetime import datetime
    
    # Remove day of week (e.g., "(Tue)")
    timestamp_clean = timestamp.split('(')[0].strip() + ' ' + timestamp.split(')')[1].strip()
    # Now it's like: '2025/12/02 17:06'
    
    # Parse the timestamp
    dt = datetime.strptime(timestamp_clean, '%Y/%m/%d %H:%M')
    
    # Convert to target format
    return dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
def add_sessions_to_memory(lightmem: LightMemory, 
                          sessions: List[List[Dict]], 
                          session_ids: List[str],
                          dates: List[str]) -> None:
    """
    Add historical sessions to the LightMemory system.
    Sessions are added turn by turn (each turn contains a user message and an assistant message).
    
    Args:
        lightmem: LightMemory instance
        sessions: List of sessions, each session contains multiple conversation turns
        session_ids: List of session IDs
        dates: List of session timestamps (will be converted to standard format)
    """
    print("Starting to add historical sessions to memory repository...")
    
    # Convert all timestamps to standard format
    print("Converting timestamps to standard format...")
    converted_dates = [convert_timestamp(date) for date in dates]
    
    # Calculate total number of turns for progress bar
    total_turns = 0
    for session in sessions:
        # Ensure first message is from user
        session_copy = session.copy()
        while session_copy and session_copy[0]["role"] != "user":
            session_copy.pop(0)
        num_turns = len(session_copy) // 2
        total_turns += num_turns
    
    progress_bar = tqdm(total=total_turns, desc="Adding turns")
    
    for session_idx, (session, session_id, date) in enumerate(zip(sessions, session_ids, converted_dates)):
        # Ensure the first message is from user
        while session and session[0]["role"] != "user":
            session.pop(0)
        
        num_turns = len(session) // 2
        
        for turn_idx in range(num_turns):
            # Extract one turn (user + assistant messages)
            turn_messages = session[turn_idx*2 : turn_idx*2 + 2]
            
            # Validate turn structure
            if len(turn_messages) < 2 or turn_messages[0]["role"] != "user" or turn_messages[1]["role"] != "assistant":
                continue
            
            # Add timestamp and speaker information to each message
            for msg in turn_messages:
                msg["time_stamp"] = date
                # Add default speaker information if not present
                if "speaker_name" not in msg:
                    msg["speaker_name"] = "User" if msg["role"] == "user" else "Assistant"
                if "speaker_id" not in msg:
                    msg["speaker_id"] = "speaker_a" if msg["role"] == "user" else "speaker_b"
            
            # Only force_segment and force_extract on the last turn of the last session
            is_last_turn = (session_idx == len(sessions) - 1 and turn_idx == num_turns - 1)
            
            # Add turn to memory system
            try:
                lightmem.add_memory(
                    messages=turn_messages,
                    METADATA_GENERATE_PROMPT=METADATA_GENERATE_PROMPT,
                    force_segment=is_last_turn,
                    force_extract=is_last_turn,
                )
                progress_bar.update(1)
            except Exception as e:
                print(f"\nWarning: Failed to add turn {turn_idx} from session {session_id}: {str(e)}")
                continue
    
    progress_bar.close()
    print("\nAll historical sessions have been added!")
    
add_sessions_to_memory(lightmem, haystack_sessions, haystack_session_ids, haystack_dates)

Starting to add historical sessions to memory repository...
Converting timestamps to standard format...


Adding turns:   0%|          | 0/273 [00:00<?, ?it/s]2025-12-06 20:23:07 - LightMemory - INFO - ========== START add_memory_20251206_202307_887960 ==========
2025-12-06 20:23:07 - LightMemory - INFO - force_segment=False, force_extract=False
2025-12-06 20:23:07 - LightMemory - INFO - [add_memory_20251206_202307_887960] Extracted 0 visual contexts
2025-12-06 20:23:08 - LightMemory - INFO - [add_memory_20251206_202307_887960] Restored visual contexts after compression
2025-12-06 20:23:08 - LightMemory - INFO - [add_memory_20251206_202307_887960] Target compression rate: 0.8
Adding turns:   0%|          | 1/273 [00:00<01:49,  2.48it/s]2025-12-06 20:23:08 - LightMemory - INFO - ========== START add_memory_20251206_202308_291645 ==========
2025-12-06 20:23:08 - LightMemory - INFO - force_segment=False, force_extract=False
2025-12-06 20:23:08 - LightMemory - INFO - [add_memory_20251206_202308_291645] Extracted 0 visual contexts
Token indices sequence length is longer than the specified maxim

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-06 20:26:30 - lightmem.factory.retriever.embeddingretriever.qdrant - INFO - Inserting 1 vectors into collection longmemeval_demo


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-06 20:26:30 - lightmem.factory.retriever.embeddingretriever.qdrant - INFO - Inserting 1 vectors into collection longmemeval_demo
2025-12-06 20:26:30 - LightMemory - INFO - [offline_update_20251206_202627_909081] Successfully inserted 17 entries to vector database
2025-12-06 20:26:30 - LightMemory - INFO - [add_memory_20251206_202621_623069] Cumulative token stats - Total API calls: 24, Total tokens: 39804
Adding turns: 100%|██████████| 273/273 [03:22<00:00,  1.35it/s]


All historical sessions have been added!


## 5. Offline update

In [8]:
lightmem.construct_update_queue_all_entries()
lightmem.offline_update_all_entries(score_threshold=0.8)

2025-12-06 20:31:47 - LightMemory - INFO - ========== START construct_queue_20251206_203147_758505 ==========
2025-12-06 20:31:47 - LightMemory - INFO - [construct_queue_20251206_203147_758505] Parameters: top_k=20, keep_top_n=10, max_workers=8
2025-12-06 20:31:47 - LightMemory - INFO - [construct_queue_20251206_203147_758505] Retrieved 263 entries from vector database
2025-12-06 20:31:47 - LightMemory - INFO - [construct_queue_20251206_203147_758505] Starting parallel queue construction with 8 workers
2025-12-06 20:32:27 - LightMemory - INFO - [construct_queue_20251206_203147_758505] Queue construction completed: 263 updated, 0 skipped, nonempty_queues=263, empty_queues=0
2025-12-06 20:32:27 - LightMemory - INFO - ========== END construct_queue_20251206_203147_758505 ==========
2025-12-06 20:32:27 - LightMemory - INFO - ========== START offline_update_all_20251206_203227_940953 ==========
2025-12-06 20:32:27 - LightMemory - INFO - [offline_update_all_20251206_203227_940953] Parameters

## 6. Retrieval and answer

In [9]:
def test_retrieval_and_answer(lightmem: LightMemory, 
                              questions: List[str], 
                              question_ids: List[str],
                              question_types: List[str],
                              question_dates: List[str],
                              answers: List[str],
                              top_k: int = 20) -> pd.DataFrame:
    """
    Perform memory retrieval, generate answers using LLM, and evaluate correctness.
    
    Args:
        lightmem: LightMemory instance
        questions: List of questions
        question_ids: List of question IDs
        question_types: List of question types
        question_dates: List of question dates
        answers: List of expected answers
        top_k: Number of memory entries to retrieve
    
    Returns:
        DataFrame containing retrieval and evaluation results
    """
    results = []
    
    print(f"Starting memory retrieval and answer generation for {len(questions)} questions...\n")
    
    # Initialize LLM for answer generation (using the same config as LightMemory)
    from openai import OpenAI
    
    llm_client = OpenAI(
        api_key=API_KEY,
        base_url=API_BASE_URL
    )
    
    # LLM for judging (can be the same)
    llm_judge = llm_client
    
    for idx, (qid, question, qtype, qdate, expected_answer) in enumerate(
        zip(question_ids, questions, question_types, question_dates, answers), 1
    ):
        print(f"\n{'='*80}")
        print(f"Question {idx}/{len(questions)} [ID: {qid}]")
        print(f"{'='*80}")
        print(f"Question: {question}")
        print(f"Question Date: {qdate}")
        print(f"Question Type: {qtype}")
        print(f"Expected Answer: {expected_answer}")
        
        try:
            # Step 1: Retrieve relevant memories
            result_string = lightmem.retrieve(question, limit=top_k)
            related_memories = [m.strip() for m in result_string.split('\n') if m.strip()]
            
            print(f"\nRetrieved {len(related_memories)} relevant memories")
            print("-" * 80)
            
            # Display first few memories
            for mem_idx, memory in enumerate(related_memories, 1):
                print(f"Memory {mem_idx}: {memory}")
            
            # Step 2: Generate answer using LLM
            print("\nGenerating answer...")
            messages = [
                {"role": "system", "content": "You can ONLY answer based on the provided memories."},
                {
                    "role": "user",
                    "content": f"Question: {question}\n\nPlease answer the question based on the following memories:\n{result_string}"
                }
            ]
            
            response = llm_client.chat.completions.create(
                model=LLM_MODEL,
                messages=messages,
                max_tokens=1024,
                temperature=0.0
            )
            
            generated_answer = response.choices[0].message.content
            print(f"\nGenerated Answer: {generated_answer}")
            
            # Step 3: Evaluate answer correctness
            print("\nEvaluating answer...")
            
            # Build evaluation prompt

            eval_prompt = f"""You are an expert evaluator. Compare the generated answer with the expected answer.
            Question: {question}
            Expected Answer: {expected_answer}
            Generated Answer: {generated_answer}

            Determine if the generated answer is correct compared to the expected answer.
            Answer only "True" or "False"."""
            
            eval_messages = [{"role": "user", "content": eval_prompt}]
            
            eval_response = llm_judge.chat.completions.create(
                model=LLM_MODEL,
                messages=eval_messages,
                max_tokens=10,
                temperature=0.0
            )
            
            eval_result = eval_response.choices[0].message.content.strip()
            correct = 1 if "true" in eval_result.lower() else 0
            
            print(f"Evaluation Result: {eval_result} ({'✓ Correct' if correct else '✗ Incorrect'})")
            
            # Record results
            results.append({
                'question_id': qid,
                'question_type': qtype,
                'question': question,
                'question_date': qdate,
                'expected_answer': expected_answer,
                'retrieved_count': len(related_memories),
                'retrieved_memories': related_memories,
                'generated_answer': generated_answer,
                'eval_result': eval_result,
                'correct': correct
            })
            
        except Exception as e:
            print(f"\nError: Processing failed - {str(e)}")
            import traceback
            traceback.print_exc()
            
            results.append({
                'question_id': qid,
                'question_type': qtype,
                'question': question,
                'question_date': qdate,
                'expected_answer': expected_answer,
                'retrieved_count': 0,
                'retrieved_memories': [],
                'generated_answer': '',
                'eval_result': '',
                'correct': 0,
                'error': str(e)
            })
    
    print(f"\n{'='*80}")
    print("Retrieval and answer generation completed!")
    print(f"{'='*80}\n")
    
    df = pd.DataFrame(results)
    
    # Print summary statistics
    if len(df) > 0 and 'correct' in df.columns:
        accuracy = df['correct'].mean() * 100
        print(f"Overall Accuracy: {accuracy:.2f}% ({df['correct'].sum()}/{len(df)})")
    
    return df



In [10]:
# Execute retrieval, answer generation, and evaluation
retrieval_results = test_retrieval_and_answer(
    lightmem, 
    questions, 
    question_ids,
    question_types,
    question_dates,
    answers, 
    top_k=20
)

2025-12-06 20:35:36 - LightMemory - INFO - ========== START retrieve_20251206_203536_594227 ==========
2025-12-06 20:35:36 - LightMemory - INFO - [retrieve_20251206_203536_594227] Query: What degree did I graduate with?
2025-12-06 20:35:36 - LightMemory - INFO - [retrieve_20251206_203536_594227] Parameters: limit=20, filters=None


Starting memory retrieval and answer generation for 1 questions...


Question 1/1 [ID: e47becba]
Question: What degree did I graduate with?
Question Date: 2023/05/30 (Tue) 23:40
Question Type: single-session-user
Expected Answer: Business Administration


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-06 20:35:36 - LightMemory - INFO - [retrieve_20251206_203536_594227] Searching vector database
2025-12-06 20:35:36 - LightMemory - INFO - [retrieve_20251206_203536_594227] Found 20 results
2025-12-06 20:35:36 - LightMemory - INFO - [retrieve_20251206_203536_594227] Formatted 20 results into output string
2025-12-06 20:35:36 - LightMemory - INFO - ========== END retrieve_20251206_203536_594227 ==========



Retrieved 20 relevant memories
--------------------------------------------------------------------------------
Memory 1: 2023-05-30T17:27:02.000 Tue User graduated with a degree in Business Administration.
Memory 2: 2023-05-30T17:27:02.000 Tue User's degree in Business Administration has helped them in their new role.
Memory 3: 2023-05-29T01:39:00.000 Mon User is writing a case study for a senior UX designer portfolio page for Grade A Steakhouse.
Memory 4: 2023-05-28T02:37:00.000 Sun User has been taking an online course on diversity and inclusion.
Memory 5: 2023-05-22T09:23:03.000 Mon User attended a singer-songwriter night at Coffee House on Main Street, which was intimate and cool.
Memory 6: 2023-05-29T01:39:00.000 Mon Grade A Steakhouse is a virtual ghost kitchen that delivers steak dinners right to the user's door.
Memory 7: 2023-05-22T09:23:01.000 Mon User got a t-shirt with The Electric Storm's album artwork from the merchandise booth.
Memory 8: 2023-05-22T09:23:01.000 Mon Use